# Chapter 2 – End-to-end Machine Learning project

In [3]:
import numpy as np
import pandas as pd

In [4]:
import pickle

with open('datasets/housing_train_prepared.pickle', 'rb') as f:
    housing_train_data = pickle.load(f)
with open('datasets/housing_train_labels.pickle', 'rb') as f2:
    housing_train_labels_pandas = pickle.load(f2)
# Testen, ob alles gut gegangen ist
housing_train_data[:10]
housing_train_labels =housing_train_labels_pandas.to_numpy()
housing_train_labels
# Umwandlung in numpy hätte eigentlich schon in Teil 2 des Projekts stattfinden sollen

array([275000., 500001., 151600., ..., 361400., 261000.,  64500.])

# Training und Bewertung von Modellen


###### Schritte während des Lern-Prozesses

In [5]:
"""
0.  Algorithmus importieren und eine Instanz anlegen

1.  Algorithmus trainieren.
    Die Funktion dazu heißt immer "fit".
    Während des Trainings erstellt der Algorithmus das Modell, 
    von dem er glaubt, dass es die Daten gut beschreibt.
    
2.  Wenn das Modell fertig ist, versucht der Mensch abzuschätzen , 
wie gut es ist
    Für Voraussagen nutzen wir die Funktion predict , 
    die jeder Algo hat
    
    a) Wir machen Voraussagen auf Beispieldaten und schätzen ab, ob das echte Label getroffen wurde.
    b) Wir berechnen eine Kennzahl für die Voraussagen .
       Hiermit wird die Qualität von allen Voraussagen  zusammengefasst.
       Grundlage ist die TRAININGS-MENGE
       
       Für die Bewertung nutzen wir Funktionen, 
           z.B. mean_absolute_error,...
       
3.  Kreuzvalidierung mit cross_val_score
        Aufteilung in n Folds
        Ergebnis: n Scores
        Zusammenfassung der n Scores: z.B Mittelwert + Standardabweichung
4.  GridSearch: suche die Parameter, 
        die einen guten Algorithmus noch verbessern
        
        berechnet den Score für jede Parameterkombination über crossval
        Schleife: 
        pro Parameterkombination und Fold ein Training und eine Bewertung
        
5.  Voting (Klasse: VotingRegressor importieren und anwenden)

6.  Bewertung des endgültigen Modells durch Berechnung einer Kennzahl. 
    Diesmal wird die Kennzahl auf der TEST-MENGE berechnet
"""
print()

###### 1__LinReg) Training mit LinearRegression

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_train_data, housing_train_labels)

LinearRegression()

###### 2a-__LinReg) Stichprobe anschauen und die Leistung des Algorithmus überprüfen

In [7]:
# Wir prüfen an 5 Original-Zeilen (den "Beispielen") , ob der Algorithmus imstande ist, 
# die Label einigermaßen richtig vorauszusagen.
# Die Originalzeilen müssen mit der Pipeline vorbereitet werden

In [8]:
#beispiele_vorbereitet=pipe.transform(beispiele_data)
np.set_printoptions(precision=2)
beispiele_vorbereitet = housing_train_data[:5]
beispiele_labels = housing_train_labels[:5]
beispiele_vorbereitet
beispiele_labels

array([275000., 500001., 151600., 238500., 379600.])

In [9]:
beispiele_predictions = lin_reg.predict(beispiele_vorbereitet)
np.set_printoptions(precision=0)
beispiele_predictions
#lin_reg.coef_

array([142133., 340560., 201755., 231829., 374943.])

In [10]:
# Vergleichen mit den echten Werten

In [11]:
beispiele_labels

array([275000., 500001., 151600., 238500., 379600.])

In [12]:
#Wie gut sind die Voraussagen?
#Als Antwort berechnen wir den prozentualen Fehler für jeden einzelnen Datensatz.
prozentuale_abweichungen = beispiele_predictions / beispiele_labels - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in prozentuale_abweichungen]))

-48.3%, -31.9%, 33.1%, -2.8%, -1.2%


######  2b__LinReg) Bewertung anhand von Kennzahlen, die aus dem gesamten Trainingsergebnis abgeleitet werden

In [13]:
housing_predictions_lin = lin_reg.predict(housing_train_data)
housing_predictions_lin[:5]
housing_train_labels[:5]

array([275000., 500001., 151600., 238500., 379600.])

In [14]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_train_labels, housing_predictions_lin )
print("Durchschnittliche Abweichung zwischen echt und vorausgesagt(MAE):", round(lin_mae),"$")

Durchschnittliche Abweichung zwischen echt und vorausgesagt(MAE): 49431 $


In [15]:
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_train_labels, housing_predictions_lin, squared=False)
print("Durchschnittliche Abweichung zwischen echt und vorausgesagt(RMSE):", round(lin_rmse),"$")

Durchschnittliche Abweichung zwischen echt und vorausgesagt(RMSE): 68460 $


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
from sklearn.metrics import mean_absolute_percentage_error
lin_mape = mean_absolute_percentage_error(
                housing_train_labels, 
                housing_predictions_lin)
print("Prozentualer Fehler",np.round(lin_mape *100),"%")

Prozentualer Fehler 28.0 %


In [17]:
# Wir probieren den nächsten Algorithmus

###### 1__Tree) Training mit DecisionTree

In [18]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_train_data, housing_train_labels)

DecisionTreeRegressor(random_state=42)

######  2b__Tre)  Bewertung anhand von Kennzahlen, die aus dem gesamten Trainingsergebnis abgeleitet werden

In [19]:
housing_predictions_tree = tree_reg.predict(housing_train_data)
housing_predictions_tree[:6]

array([275000., 500001., 151600., 238500., 379600., 203300.])

In [20]:

tree_mae = mean_absolute_error(housing_train_labels, housing_predictions_tree)
tree_mae

0.0

In [21]:
housing_predictions = tree_reg.predict(housing_train_data)
tree_rmse = mean_squared_error(housing_train_labels, housing_predictions, squared=False)
tree_rmse

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.0

In [22]:
# Haben wir nun den perfekten Algorithmus gefunden ???

## 3. Bessere Bewertungsgrundlage: Cross-Validation (Kreuz-Evaluation)

In [23]:
# Der Trainingsscore  des DecisionTrees ist deutlich besser als der Crossvalscore.
# Das nennt man OVERFITTING .
# Das muss geändert werden, ehe man dem Score vertraut. 
# Wenn das nicht zu ändern ist, taugt der Algorithmus nichts für diese Daten.

In [24]:
from sklearn.model_selection import cross_val_score,cross_validate

### 3 Scoring-Verfahren für den DecisionTree

#### DecisionTree ________Scoring-Verfahren : root mean squared error

In [25]:
tree_rmses = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
print("Evaluierungs-Scores DecisionTree, Verfahren RMSE")
tree_rmses
#crossval_score(instanz_des_algorithmus, 
#               trainingsdaten,trainingslabel, 
#               scoring=Bewertungsmaßstab,
#               cv = Anzahl_der_folds)

Evaluierungs-Scores DecisionTree, Verfahren RMSE


array([80793., 79032., 80603., 76257., 78667., 75992., 78942., 76809.,
       81157., 83162.])

In [26]:
print("Mittelwert aller Teil-RMSE-Scores (DecisionTree):",round(tree_rmses.mean()),
      "_____ Standardabweichung:",round(tree_rmses.std()))
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-RMSE-Scores (DecisionTree): 79141 _____ Standardabweichung: 2215


#### DecisionTree__________Scoring-Verfahren :Mean absolute Error

In [27]:
tree_maes = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_error", cv=10)
tree_maes

array([54139., 52184., 53324., 51204., 52121., 51197., 52693., 51786.,
       54221., 54365.])

In [28]:
print("Mittelwert aller Teil-MAE-Scores (DecisionTree):",round(tree_maes.mean()),
      "$_____ Standardabweichung der Fehler:",round(tree_maes.std()),"$")
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-MAE-Scores (DecisionTree): 52724 $_____ Standardabweichung der Fehler: 1161 $


#### DecisionTree_________Scoring-Verfahren: Mean absolute percentage error

In [29]:
tree_mapes = -cross_val_score(tree_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_percentage_error", cv=10)
np.round(tree_mapes *100)

array([29., 30., 29., 28., 29., 29., 28., 28., 29., 29.])

In [30]:
print("Mittelwert aller Teil-MAPE-Scores (DecisionTree):",round(tree_mapes.mean()*100),
      "%_____ Standardabweichung der Fehler:",round(tree_mapes.std()*100,2),"%")
#pd.DataFrame(tree_rmses).describe()

Mittelwert aller Teil-MAPE-Scores (DecisionTree): 29 %_____ Standardabweichung der Fehler: 0.55 %


### Zum Vergleich: Die Scores der linearen Regression

##### Lineare Regression____ Scoring

In [31]:
lin_rmses = -cross_val_score(lin_reg, housing_train_data, housing_train_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
print(f"Mittelwert aller Teil-RMSE-Scores  (LineareRegression): {round(lin_rmses.mean())} $",
      f"_____ Standardabweichung: {round(lin_rmses.std())} $")
#pd.Series(lin_rmses).describe()


Mittelwert aller Teil-RMSE-Scores  (LineareRegression): 68703 $ _____ Standardabweichung: 3684 $


In [32]:
lin_mape_cv = -cross_val_score(lin_reg, housing_train_data, housing_train_labels,
                              scoring="neg_mean_absolute_percentage_error", cv=10)
print("Mittelwert aller Teil-MAPE-Scores  (LineareRegression):",round(lin_mape_cv.mean()*100),
      "%_____ Standardabweichung:",round(lin_mape_cv.std()*100))

Mittelwert aller Teil-MAPE-Scores  (LineareRegression): 29 %_____ Standardabweichung: 1


In [33]:
# Der Trainingsscore der Linearen Regression stimmt zwar ungefähr mit der Crossval-Score über ein.
# Aber der Score ist schlecht.
# Das nennt man UNDER-FITTING.
# Wir vertrauen diesem Algorithmus ebenfalls nicht.

### Nächster Versuch: Wir trainieren und evaluieren den Algorithmus RandomForest.
**Warning:** the following cell may take a few minutes to run:

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42, n_jobs = -1)
forest_rmses = -cross_val_score(forest_reg, housing_train_data, housing_train_labels,
                                scoring="neg_root_mean_squared_error", cv=5)

In [35]:
print(f"Durchschnitt der CV-Scores(RMSE): {round(forest_rmses.mean())} $", 
      f"_____ Standardabweichung: {round(forest_rmses.std())} $")
#pd.Series(forest_rmses).describe()

Durchschnitt der CV-Scores(RMSE): 55495 $ _____ Standardabweichung: 2139 $


In [36]:
forest_mapes = -cross_val_score(forest_reg, housing_train_data, housing_train_labels,
                                scoring="neg_mean_absolute_percentage_error", cv=5)
forest_mapes *100

array([22., 21., 21., 21., 22.])

In [37]:
print(f"Durchschnitt der CV-Scores(MAPE): {round(forest_mapes.mean()*100)} %",
      f"_____ Standardabweichung: {round(forest_mapes.std()*100)} %")
#(pd.Series(forest_mapes)*100).describe()

Durchschnitt der CV-Scores(MAPE): 21 % _____ Standardabweichung: 0 %


In [38]:
# Dieser Algorithmus liefert schon bessere Werte.
# Wir versuchen demnächst, die Bewertung weiter zu verbessern, indem wir nicht mehr mit den Default-Parametern arbeiten.
# Wir wollen die besten Varianten für die Parameter herausfinden.

### Nächster Versuch: Wir trainieren und evaluieren den Algorithmus "Support Vector Machine".

Warnung: die Programmausführung dauert ziemlich lang.

In [39]:
"""
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

svr = SVR()
svr.fit(housing_train_data, housing_train_labels)
svr_train_predictions = svr.predict(housing_train_data)
print(round(
    mean_squared_error(
        housing_train_labels,svr_train_predictions,squared=False)))

#svr_rmses = -cross_val_score(svr, housing_train_data, housing_train_labels,
#                               scoring="neg_root_mean_squared_error", cv=5)
# Ergebnis rmse =111_000 $
"""
print()

# Fine-Tuning: Die Modellparameter werden optimiert

## Grid Search mit RMSE

Warnung: die Programmausführung  eines Gridsearch dauert sehr lang.

In [40]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    { 'max_features': [2,4,6,8],
      'n_estimators': [3,10,30]}# oder ...[10,100,1000,10000]
#,
#    {'bootstrap': [False],
#     'max_features': [2, 3,4],
#     'n_estimators': [3, 10]}
]
#param_grid enthält eine Dictionary, das ausgewählte Parameter von der Klasse RandomForest zeigt.
# Diese Parameter heißen HYPER-PARAMETER
# Die Listen, die hinter den Parametern stehen , enthalten mögliche Werte der Parameter.

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                           param_grid, 
                           cv=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs = -1)
grid_search.fit(housing_train_data, housing_train_labels)
# Das grid_search-Objekt führt ein fit für jede Parameter-kombination aus und macht auch eine Kreuz-Evaluierung(Crossval)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             scoring='neg_root_mean_squared_error')

In [41]:
# Anmerkung:
# Man kann auch einen Gridsearch über alle Parameter einer Pipeline machen.
# Die Liste der Parameter kann man herausfinden mit :`full_pipeline.get_params().keys()`

In [42]:
# Die grid_search-Instanz findet die besten Parameter
grid_search.best_params_
# Anmerkung : der Score wird hier sehr schlecht, weil die Parameter im Gridsearch ungünstig gewählt waren.

{'max_features': 6, 'n_estimators': 30}

In [43]:
# Die grid_search-Instanz kann auch einen Estimator erstellen, der die besten Parameter schon enthält.
final_model=grid_search.best_estimator_
final_model

RandomForestRegressor(max_features=6, n_estimators=30, random_state=42)

Let's look at the score of each hyperparameter combination tested during the grid search:

In [44]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False,
                   inplace=True)
cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,1.728689,0.036784,0.028316,0.001392,6,30,"{'max_features': 6, 'n_estimators': 30}",-56857.581083,-54625.884647,-57862.322240,-56448.595990,1352.549119,1
11,1.994690,0.025357,0.024461,0.000454,8,30,"{'max_features': 8, 'n_estimators': 30}",-57228.280492,-54636.003037,-58227.505950,-56697.263160,1513.540628,2
5,1.341903,0.029166,0.033468,0.003749,4,30,"{'max_features': 4, 'n_estimators': 30}",-57398.690357,-54683.434974,-58167.944819,-56750.023383,1494.663585,3
2,0.871310,0.031122,0.035001,0.003862,2,30,"{'max_features': 2, 'n_estimators': 30}",-58781.281451,-56251.458690,-59099.822998,-58044.187713,1274.303782,4
7,0.689321,0.017204,0.011362,0.000848,6,10,"{'max_features': 6, 'n_estimators': 10}",-58816.615361,-57001.986214,-60259.614440,-58692.738672,1332.802684,5
10,0.864814,0.033630,0.014265,0.000596,8,10,"{'max_features': 8, 'n_estimators': 10}",-59588.877950,-56876.873120,-59627.662033,-58697.804368,1287.690182,6
4,0.526977,0.012627,0.014355,0.000855,4,10,"{'max_features': 4, 'n_estimators': 10}",-59859.880443,-57388.998535,-60533.058626,-59260.645868,1351.687839,7
1,0.350860,0.021402,0.015701,0.001548,2,10,"{'max_features': 2, 'n_estimators': 10}",-62204.079209,-59150.873414,-61618.552097,-60991.168240,1323.058146,8
9,0.303681,0.008296,0.005511,0.000817,8,3,"{'max_features': 8, 'n_estimators': 3}",-66548.593005,-63734.919926,-66177.140643,-65486.884525,1248.072963,9
6,0.268302,0.006110,0.005512,0.001084,6,3,"{'max_features': 6, 'n_estimators': 3}",-66502.943592,-64337.179018,-67060.173034,-65966.765215,1174.532401,10


In [45]:
# extra code – these few lines of code just make the DataFrame look nicer
cv_res = cv_res[[
                 "param_n_estimators","param_max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
#score_cols = ["split0", "split1", "split2", "mean_test_mae"]
#cv_res.columns = ["n_estimators","max_features"] + score_cols
#cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)

cv_res

,param_n_estimators,param_max_features,split0_test_score,split1_test_score,split2_test_score,mean_test_score
8,30,6,-56857.581083,-54625.884647,-57862.322240,-56448.595990
11,30,8,-57228.280492,-54636.003037,-58227.505950,-56697.263160
5,30,4,-57398.690357,-54683.434974,-58167.944819,-56750.023383
2,30,2,-58781.281451,-56251.458690,-59099.822998,-58044.187713
7,10,6,-58816.615361,-57001.986214,-60259.614440,-58692.738672
10,10,8,-59588.877950,-56876.873120,-59627.662033,-58697.804368
4,10,4,-59859.880443,-57388.998535,-60533.058626,-59260.645868
1,10,2,-62204.079209,-59150.873414,-61618.552097,-60991.168240
9,3,8,-66548.593005,-63734.919926,-66177.140643,-65486.884525
6,3,6,-66502.943592,-64337.179018,-67060.173034,-65966.765215


In [46]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.147120,0.013370,0.006013,0.000827,2,3,"{'max_features': 2, 'n_estimators': 3}",-71289.386834,-67404.853124,-69749.671586,-69481.303848,1597.167591,12
1,0.350860,0.021402,0.015701,0.001548,2,10,"{'max_features': 2, 'n_estimators': 10}",-62204.079209,-59150.873414,-61618.552097,-60991.168240,1323.058146,8
2,0.871310,0.031122,0.035001,0.003862,2,30,"{'max_features': 2, 'n_estimators': 30}",-58781.281451,-56251.458690,-59099.822998,-58044.187713,1274.303782,4
3,0.196648,0.008916,0.005511,0.000816,4,3,"{'max_features': 4, 'n_estimators': 3}",-67349.427163,-65712.751079,-69022.787097,-67361.655113,1351.344208,11
4,0.526977,0.012627,0.014355,0.000855,4,10,"{'max_features': 4, 'n_estimators': 10}",-59859.880443,-57388.998535,-60533.058626,-59260.645868,1351.687839,7
5,1.341903,0.029166,0.033468,0.003749,4,30,"{'max_features': 4, 'n_estimators': 30}",-57398.690357,-54683.434974,-58167.944819,-56750.023383,1494.663585,3
6,0.268302,0.006110,0.005512,0.001084,6,3,"{'max_features': 6, 'n_estimators': 3}",-66502.943592,-64337.179018,-67060.173034,-65966.765215,1174.532401,10
7,0.689321,0.017204,0.011362,0.000848,6,10,"{'max_features': 6, 'n_estimators': 10}",-58816.615361,-57001.986214,-60259.614440,-58692.738672,1332.802684,5
8,1.728689,0.036784,0.028316,0.001392,6,30,"{'max_features': 6, 'n_estimators': 30}",-56857.581083,-54625.884647,-57862.322240,-56448.595990,1352.549119,1
9,0.303681,0.008296,0.005511,0.000817,8,3,"{'max_features': 8, 'n_estimators': 3}",-66548.593005,-63734.919926,-66177.140643,-65486.884525,1248.072963,9


## Gridsearch mit MAPE

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    { 'max_features': [2,4,6,8],
      'n_estimators': [3,10,30]}# oder ...[10,100,1000,10000]
#,
#    {'bootstrap': [False],
#     'max_features': [2, 3,4],
#     'n_estimators': [3, 10]}
]
#param_grid enthält eine Dictionary, das ausgewählte Parameter von der Klasse RandomForest zeigt.
# Diese Parameter heißen HYPER-PARAMETER
# Die Listen, die hinter den Parametern stehen , enthalten mögliche Werte der Parameter.

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), 
                           param_grid, 
                           cv=3,
                           scoring='neg_mean_absolute_percentage_error',
                           n_jobs = -1)
grid_search.fit(housing_train_data, housing_train_labels)
# Das grid_search-Objekt führt ein fit für jede Parameter-kombination aus und macht auch eine Kreuz-Evaluierung(Crossval)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             scoring='neg_mean_absolute_percentage_error')

In [48]:
# Die grid_search-Instanz findet die besten Parameter
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [49]:
print( "Score der besten Parameterkombination",-np.round(grid_search.best_score_ *100,2),"%")

Score der besten Parameterkombination 22.09 %


In [50]:
# Die grid_search-Instanz kann auch einen Estimator erstellen, der die besten Parameter schon enthält.
final_model=grid_search.best_estimator_
final_model

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

## Evaluate Your System on the Test Set

In [51]:
with open('datasets/housing_test_prepared.pickle', 'rb') as f3:
    housing_test_data = pickle.load(f3)
print("Testdaten",housing_test_data)

with open('datasets/housing_test_labels.pickle', 'rb') as f3:
    housing_test_labels = pickle.load(f3)
print("Testdaten",housing_test_labels)

Testdaten [[-0.  2.  1. ...  0.  0.  0.]
 [ 0. -0.  1. ...  0.  0.  0.]
 [-1. -0. -0. ...  0.  0.  0.]
 ...
 [ 3. -2. -4. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  1.]
 [-0. -0.  0. ...  0.  0.  0.]]
Testdaten 13669    288000.0
14726    277700.0
3856      93600.0
6635     178700.0
9811     141600.0
           ...   
8713     252000.0
7565     298400.0
7025     181800.0
18084    259400.0
17316    110000.0
Name: median_house_value, Length: 4128, dtype: float64


In [56]:
final_predictions = final_model.predict(housing_test_data)

from sklearn.metrics import mean_squared_error

# Calculate RMSE
final_rmse = np.round(mean_squared_error(housing_test_labels, final_predictions, squared=False), 2)
print(final_rmse, "$")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [4128, 16512]

In [54]:
from sklearn.metrics import mean_absolute_percentage_error
final_mape = np.round(mean_absolute_percentage_error(housing_test_labels, final_predictions)*100)
print(final_mape,"%")

ValueError: Found input variables with inconsistent numbers of samples: [4128, 16512]

In [ ]:
# Anmerkung: Der Score ist hier so schlecht, 
# weil die Testmenge offenbar eine andere STruktur hat asl die Trainingsmenge

Nur für Leute mit guten Statistik kenntnissen:
Wir können ein 95% Konfidenz-Intervall für die test-RMSE konstruieren

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - housing_test_labels) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([53415., 58080.])

In [ ]:
# extra code – shows how to compute a confidence interval for the RMSE
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)# T-Verteilung !
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

(53414.62618182245, 58080.038037041086)

Falls der Chef das Modell gut genug findet, kann es ab jetzt im Unternehmen eingesetzt werden.